# Allrecipes.com Cocktail Recipe Scraping

-  Reference: 
    -  https://nycdatascience.com/blog/student-works/recipes-scraping-top-20-recipes-allrecipes/
-  Selenium is used because the webpages utilize AJAX, combined with beautifulsoup the speed of scraping seems to be improved 

## Import packages and define functions

In [1]:
import os
import sys
import bs4
import requests
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

######### see Lab3 ###############
def get_text_from_elements(elements):
    """Uses list comprehension to parse out the cleaned text strings from a list of
    elements returned from a BeautifulSoup selection.

    Arguments:
        elements {list} -- list of elements returned from a BeautifulSoup selection

    Returns:
        list -- list of cleaned text contained within the element list
    """
    return [e.text.strip() for e in elements]


## Choosing cocktails with different base liquors

In [2]:
# get the URL for different categories of drinks, no other way than just to explore the website
rumURL = "https://www.allrecipes.com/recipes/1741/drinks/cocktails/rum-drinks/"
ginURL = "https://www.allrecipes.com/recipes/1739/drinks/cocktails/gin-drinks/"
bourbonURL = "https://www.allrecipes.com/recipes/16957/drinks/cocktails/bourbon-drinks/"
champURL = "https://www.allrecipes.com/recipes/14975/drinks/cocktails/champagne-drinks/"
blendedURL = "https://www.allrecipes.com/recipes/17651/drinks/cocktails/blended-cocktails/"
tequilaURL = "https://www.allrecipes.com/recipes/1742/drinks/cocktails/tequila-drinks/"
vodkaURL = "https://www.allrecipes.com/recipes/1743/drinks/cocktails/vodka-drinks/"
whiskeyURL = "https://www.allrecipes.com/recipes/1744/drinks/cocktails/whiskey-drinks/"

baseURL = (rumURL, ginURL, bourbonURL, champURL, blendedURL, tequilaURL, vodkaURL, whiskeyURL)

#print(baseURL)


## Storing all recipeID into a list for scraping

In [3]:
# grab all the recipe IDs from the list of URLs
recipeID = list()
browser = webdriver.Chrome()
for url in baseURL:
    browser.get(url)
    #data = r.text
    time.sleep(5)
    soup = bs(browser.page_source, 'html.parser')
    elements = soup.select('article.fixed-recipe-card > a')
    for e in elements:
        drink_id = e.get("data-id")
    #ls = [e.get_attribute('data-id') for e in browser.find_elements_by_class_name('favorite')]
    #print([e.get_attribute('data-id') for e in browser.find_elements_by_class_name('favorite')])
    #print(browser.find_elements_by_css_selector('ar-save-item.favorite'))
        recipeID.append(drink_id)
    
#print(len(recipeID))


## Web Scraping individual webpages for Cocktail recipes

In [4]:
#Disabling JavaScript loading to speed up the process
chrome_options = Options()
chrome_options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

dict = {}
for drink_id in recipeID:
    #print(drink_id)
    url = "https://www.allrecipes.com/recipe/"+ drink_id
    browser.get(url)
    time.sleep(5)
    soup = bs(browser.page_source, 'html.parser')
    name = get_text_from_elements(soup.select("h1.recipe-summary__h1"))
    if len(name) >= 0:
        #print(str(name))
        #elements = soup.select("section.recipe-summary > h1")
        ingre = soup.select("span.recipe-ingred_txt.added")
        ls=get_text_from_elements(ingre)
        if len(ls) >= 2:
            ls.pop()
            ls.pop()
            #print(ls)
            dict[str(name)]=ls
#print(dict)

##  Save to an intermediary .csv file for cleaning

In [6]:
#print(len(dict))
# Now, put them in a csv file and clean it later
df = pd.DataFrame.from_dict(dict, orient="index")
df.to_csv("allrecipes_raw.csv")

writer = pd.ExcelWriter('Workbook_Prototype.xlsx', engine='xlsxwriter')
df = pd.read_csv('allrecipes_raw.csv')
df.to_excel(writer, sheet_name='allrecipes_raw')
writer.save()
